# Extract top 1M pages
From here: https://dumps.wikimedia.org/other/pageviews/2024/2024-12/


In [14]:
from pathlib import Path
import os 
CWD = Path(os.getcwd())
BASEDIR = CWD.parent
asset_dir = BASEDIR / 'assets'
stats_f = asset_dir/ 'pageviews-20241201-000000'
stats_f.exists()

True

## Example from stats file:

project page_title views bytes

```

en.m Fertile_material 1 0
en.m Fertilisation 3 0
en.m Fertiliser 1 0
en.m Fertilisers_and_Chemicals_Travancore 1 0
en.m Fertility 10 0
en.m Fertility_and_intelligence 4 0
en.m Fertility_and_religion 2 0
en.m Fertility_awareness 2 0
en.m Fertility_clinic 1 0
en.m Fertility_fraud 5 0
en.m Fertility_goddess 1 0
en.m Fertility_in_art 2 0
en.m Fertility_preservation 1 0

```

In [2]:
import pandas as pd

# Replace 'filename.txt' with your actual file path
df = pd.read_csv(stats_f, sep=' ', header=None, names=['project', 'page_title', 'views', 'bytes'])

print(df.head())


  project                              page_title  views  bytes
0     NaN                                       -      1      0
1     NaN             Category:Project_concept/ss      1      0
2     NaN                      File:En_obs_v8.pdf      1      0
3     NaN             File:Wikifunctions-logo.svg      1      0
4     NaN  Special:MyLanguage/Wikifunctions:About      1      0


In [18]:
english_mask = (df["project"] == "en") | (df["project"] == "en.m")
english_df = df[english_mask]


english_df.head()

english_df = english_df.groupby('page_title', as_index=False).agg({'views': 'sum', 'bytes': 'sum'})


# sort by views
english_df = english_df.sort_values(by='views', ascending=False)
# combine views from en and en.m

# for page ttles swap _ wit " "
english_df['page_title'] = english_df['page_title'].str.replace('_', ' ')

# drop bytes col
english_df = english_df.drop(columns=['bytes'])
# save to asset_dir
output_f = asset_dir / 'english_pageviews.csv'
english_df.to_csv(output_f, index=False)
print(f"saved to {output_f}")
print(english_df.shape)
english_df.head(20)


saved to /n/home04/rrinberg/code/wiki-rag/assets/english_pageviews.csv
(1678755, 2)


,page_title,views
1387629,Special:Search,207123
974995,Main Page,175514
1424275,Survivor Series: WarGames (2024),36908
1632499,Wikipedia:Featured pictures,20938
1630391,Wicked (2024 film),17021
1630398,Wicked (musical),14723
231035,Bob Bryar,13799
1046373,Moana 2,11342
334433,Charles Kushner,11295
623981,Gladiator II,11059
